<a href="https://colab.research.google.com/github/SanjayD23/Alphabetical-Hand-gesture-recognition-using-deep-leaning/blob/main/Alphabetical_Hand_gesture_recognition_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
data_path = "/content/drive/MyDrive"
os.chdir(data_path)
os.listdir(data_path)

['Scan2021-07-06_114546.jpg',
 'Scan2021-07-06_115401.jpg',
 'Scan2021-07-06_115444.jpg',
 'ভালোবাসি.pdf',
 'IMG_20240411_183809.jpg',
 'important.pdf',
 'DPS.zip',
 'IMG_20241025_131739.pdf',
 'IMG_20241024_171524.pdf',
 'Classroom',
 '1000089498.pdf',
 '1000089490 (1).pdf',
 '1000089502.pdf',
 '1000089490.pdf',
 'IMG-20240411-WA0000.jpg',
 'Colab Notebooks',
 'Screenshot 2025-07-11 130103.png',
 'Secondary.pdf',
 'Higher Secondary.pdf',
 'Voter Card.pdf',
 'Kalyani Registration.pdf',
 'images.csv',
 'images.gsheet',
 'alphaset']

OSError: [Errno 95] Operation not supported: '../dj'

In [7]:
import cv2 as cv
import os
import numpy as np

os.mkdir(os.path.join(data_path, 'gray_image'))

for i in os.listdir(os.path.join(data_path, 'alphaset')):
    if i in ['Alphabetical Hand gesture recognition using deep learning.ipynb', '.ipynb_checkpoints', 'images.csv']:
        continue

    class_gray_dir = os.path.join(data_path, 'gray_image', i)
    os.mkdir(class_gray_dir)
    for j in os.listdir(os.path.join(data_path, 'alphaset', i)):
        image_path = os.path.join(data_path, 'alphaset', i, j)
        image = cv.imread(image_path)
        if image is None:
            continue

        gray_image = np.mean(image, axis=2).astype(np.uint8)
        print(j)
        print(gray_image)

        save_path = os.path.join(class_gray_dir, f'gray_{j}')
        cv.imwrite(save_path, gray_image)


Streaming output truncated to the last 5000 lines.
A_0.jpg
[[40 40 43 ... 48 47 44]
 [39 41 45 ... 50 48 48]
 [45 47 51 ... 54 53 52]
 ...
 [25 28 34 ... 32 29 31]
 [26 27 30 ... 32 28 30]
 [30 27 28 ... 33 28 31]]
A_15.jpg
[[31 31 33 ... 25 24 24]
 [29 30 33 ... 25 23 20]
 [31 33 36 ... 31 28 26]
 ...
 [27 28 32 ... 21 23 23]
 [27 27 28 ... 22 25 21]
 [27 26 27 ... 25 21 24]]
A_118.jpg
[[34 33 34 ... 33 32 35]
 [31 31 34 ... 36 33 35]
 [33 35 40 ... 42 37 37]
 ...
 [29 33 39 ... 16 15 16]
 [30 34 38 ... 13 16 13]
 [32 35 40 ... 17 15 20]]
A_140.jpg
[[36 33 34 ... 28 29 32]
 [33 34 35 ... 31 29 32]
 [35 37 41 ... 36 34 34]
 ...
 [28 29 34 ... 15 12 14]
 [29 30 32 ... 15 17 13]
 [31 31 33 ... 18 14 17]]
A_13.jpg
[[32 30 33 ... 23 24 27]
 [28 29 32 ... 25 24 23]
 [29 30 35 ... 29 27 25]
 ...
 [25 25 30 ... 22 24 23]
 [25 22 26 ... 22 24 18]
 [27 22 25 ... 24 22 25]]
A_181.jpg
[[41 42 44 ... 45 48 45]
 [42 43 46 ... 48 51 46]
 [47 47 49 ... 58 61 57]
 ...
 [22 23 26 ... 31 30 32]
 [23 21 

KeyboardInterrupt: 

In [ ]:
import cv2 as cv
import os
import numpy as np
import pandas as pd

import cv2 as cv
import pandas as pd

data = []
image = cv.imread('A/A_0.jpg')
image = image.flatten()
num_pixels = image.size

columns = ['pixel' + str(k) for k in range(num_pixels)] + ['label']
d = pd.DataFrame([], columns=columns)
for i in os.listdir(data_path):
    if i in ['Alphabetical Hand gesture recognition using deep learning.ipynb', '.ipynb_checkpoints', 'images.csv']:
        continue
    for j in os.listdir(os.path.join(data_path, i)):
        image = cv.imread(os.path.join(data_path, i, j))
        if image is None:
            continue
        image = image.flatten()
        label = ord(i)
        d.loc[len(d)] = np.append(image, label)

d.to_csv('images1.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
images=pd.read_csv('images.csv')
print(images.head(10))

                    pixels label
0  [20 81 17 ...  7 33  9]     A
1  [14 74 13 ... 11 37 13]     A
2  [23 81 22 ... 23 46 24]     A
3  [20 82 22 ... 19 43 19]     A
4  [16 76 15 ... 11 38 12]     A
5  [12 72 11 ...  7 33  9]     A
6  [12 72 11 ... 10 36 12]     A
7  [15 75 14 ...  6 32  8]     A
8  [17 78 14 ...  9 36 10]     A
9  [14 74 13 ...  8 35  9]     A


In [ ]:

arr=np.array(images['pixels'][0])
# print(type(arr))
# for i in range(arr.size):
#   print(arr[i])

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [ ]:
import re
import ast
all_images_average_pixels = []

for index, row in images.iterrows():
    pixel_str = row['pixels']
    pixel_values_str = re.sub(r'[\[\]...]', '', pixel_str).split()
    pixel_values = [int(val) for val in pixel_values_str]

    average_of_threes = [
        sum(pixel_values[i:i+3]) / 3
        for i in range(0, len(pixel_values), 3)
        if len(pixel_values[i:i+3]) == 3
    ]

    all_images_average_pixels.append(average_of_threes)

images['average_pixels'] = all_images_average_pixels

print("Calculated average of every three consecutive pixel values for all images.")
display(images.tail())

images.to_csv('images_with_avg_pixels.csv', index=False)
print("DataFrame with average pixels saved to 'images_with_avg_pixels.csv'")

images_loaded = pd.read_csv('images_with_avg_pixels.csv')
images_loaded['average_pixels'] = images_loaded['average_pixels'].apply(ast.literal_eval)

print("CSV loaded back with 'average_pixels' as lists.")
display(images_loaded.tail())


Calculated average of every three consecutive pixel values for all images.


,pixels,label,average_pixels
26058,[ 9 67 9 ... 10 34 10],Z,"[28.333333333333332, 18.0]"
26059,[10 68 10 ... 9 33 9],Z,"[29.333333333333332, 17.0]"
26060,[10 68 9 ... 11 35 11],Z,"[29.0, 19.0]"
26061,[ 4 61 6 ... 8 31 9],Z,"[23.666666666666668, 16.0]"
26062,[13 68 11 ... 11 36 10],Z,"[30.666666666666668, 19.0]"


DataFrame with average pixels saved to 'images_with_avg_pixels.csv'
CSV loaded back with 'average_pixels' as lists.


,pixels,label,average_pixels
26058,[ 9 67 9 ... 10 34 10],Z,"[28.333333333333332, 18.0]"
26059,[10 68 10 ... 9 33 9],Z,"[29.333333333333332, 17.0]"
26060,[10 68 9 ... 11 35 11],Z,"[29.0, 19.0]"
26061,[ 4 61 6 ... 8 31 9],Z,"[23.666666666666668, 16.0]"
26062,[13 68 11 ... 11 36 10],Z,"[30.666666666666668, 19.0]"


In [ ]:
import pandas as pd
l=['a', 'b', 'c', 'd']
d=pd.DataFrame([l, l], columns=('a', 'b', 'c', 'd'))
d.to_csv('dj.csv', index=False)

,a,b,c,d
0,a,b,c,d
1,a,b,c,d
2,1,2,3,4
